In [1]:
import numpy as np
from astropy.visualization import hist
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle
from tqdm import tqdm_notebook as tqdm
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import fits
from scipy.stats import gaussian_kde
from scipy import interpolate

import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

import keras
from tensorflow.keras.models import Model, Sequential, load_model # basic class for specifying and training a neural network
from tensorflow.keras.layers import Convolution1D, Dense, Dropout, Flatten, Input, Dense  # the two types of neural network layer we will be using
from vis.visualization import visualize_saliency
from vis.utils import utils

from sklearn.model_selection import KFold

#fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
tbcolor = '#4B0082'
taucolor = '#FF8C00'

fontScale = 16
params = {'axes.labelsize': fontScale,
        'axes.titlesize': fontScale,
        'axes.grid': False,
        'axes.linewidth': 2,
        'font.size': fontScale,
        'legend.fontsize': fontScale,
        'xtick.labelsize': fontScale,
        'ytick.labelsize': fontScale,
        'xtick.major.size': 4,
        'xtick.major.width': 1.2,
        'xtick.minor.size': 3,
        'xtick.minor.width': 1.2,
        'ytick.major.size': 4,
        'ytick.major.width': 1.2,
        'ytick.minor.size': 3,
        'ytick.minor.width': 1.2,
}
# rc('text', usetex=False)
plt.rcParams.update(params)

In [10]:
# Download external data:
download = False
if download:
    # Training and test spectra
    !wget -O data/training_data_kok14_galfa.pickle https://www.dropbox.com/s/kxbiptormcsr9ya/training_data_kok14_galfa.pickle?dl=1 --no-check-certificate
    # fCNM CNN maps and uncertainties
    !wget -O data/fcnm_zea.fits https://www.dropbox.com/s/ibzeeotk6ucqup0/fcnm_zea.fits?dl=1 --no-check-certificate
    # RHI CNN maps and uncertainties
    !wget -O data/rhi_zea.fits https://www.dropbox.com/s/9l85ezgr62jiu8z/rhi_zea.fits?dl=1 --no-check-certificate
    # Observed catalog
    !wget -O data/catalogged_verification.fits https://www.dropbox.com/s/5ufmhhksk2gdzbp/catalogged_verification.fits?dl=1 --no-check-certificate
        
    data_dir = './data/'
else:
    data_dir = './cnn_temp/data/'

training_data_file = data_dir + 'training_data_kok14_galfa.pickle'
fcnm_cnn_file = data_dir + 'fcnm_zea.fits'
rhi_cnn_file = data_dir + 'fcnm_rhi.fits'
observed_catalog_file = data_dir + 'catalogged_verification.fits'

In [12]:
data = pickle.load(open(training_data_file, 'rb'))

# Synthetic training data
X_train = data['X_train']
Y_train = data['Y_train']

# Observed test data
X_test = data['X_test'] 
Y_test = data['Y_test'] 

# Re-shape network inputs
X_train = X_train.reshape(X_train.shape[0],  X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0],  X_test.shape[1], 1)

In [13]:
print(np.shape(X_train))
print(np.shape(X_test))

(29041, 414, 1)
(65, 414, 1)


In [14]:
batch_size = 16
num_epochs = 12

cnn = Sequential((
    Convolution1D(8, 6, activation='relu', input_shape=X_train.shape[1:3],name = 'firstconv'),
    Convolution1D(8, 40, activation='relu', name = 'midconv'),
    Flatten(),
    Dropout(0.3),
    Dense(2, activation='linear', name = 'preds'),     # For binary classification, change the activation to 'sigmoid'
))
cnn.compile(loss='mse', optimizer='adam')

cnn.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1,  validation_split=0.3)

Train on 20328 samples, validate on 8713 samples
Epoch 1/12
16208/20328 [======================>.......] - ETA: 2s - loss: 0.0285

KeyboardInterrupt: 

In [ ]:
# Split out the validation data from the training data:
n = np.shape(X_train)[0]
validation_split = 0.3
X_validate = X_train[:int(n*(1-validation_split))]
Y_validate = Y_train[:int(n*(1-validation_split))]

validationScore = cnn.evaluate(X_validate, Y_validate, verbose=0)
print('Validation Score', validationScore)

testScore = cnn.evaluate(X_test, Y_test, verbose=0)
print('(observed) Test Score', testScore)

Y_predicted = cnn.predict(X_validate)

cnn.save(training_data_file.replace('.pickle', '_trained.hdf5')

Now with k-fold cross validation

In [ ]:
batch_size = 16
num_epochs = 12

# define 10-fold cross validation test harness
kf = KFold(n_splits=10)
kf.get_n_splits(X_train)
 
cvscores = []
for i, (train_index, val_index) in tqdm(enumerate(kf.split(X_train, Y_train.argmax(1)))):
    x_train_kf, x_val_kf = X_train[train_index], X_train[val_index]
    y_train_kf, y_val_kf = Y_train[train_index], Y_train[val_index]

    cnn_kf = Sequential((
        Convolution1D(8, 6, activation='relu', input_shape=x_train_kf.shape[1:3]),
        Convolution1D(8, 40, activation='relu'),
        Flatten(),
        Dropout(0.3),
        Dense(2, activation='linear'),     
    ))
    cnn_kf.compile(loss='mse', optimizer='adam', metrics=['mae','acc'])
    
    cnn_kf.fit(x_train_kf, y_train_kf, epochs=num_epochs, batch_size=batch_size, verbose=1)
    
    # evaluate the model
    scores = cnn_kf.evaluate(x_val_kf, y_val_kf, verbose=0)
    print("%s: %.2f%%" % (cnn_kf.metrics_names[1], scores[1]*100))
    
    cvscores.append(scores[1] * 100)

In [ ]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
# Select the output layer
layer_idx = utils.find_layer_idx(cnn, 'preds')

fig = plt.figure(0,[24,18])

# Compute the saliency of the output layer for the observed test set as input
for k in tqdm(range(np.shape(Y_test)[0])):
    grads_fcnm = visualize_saliency(cnn, layer_idx, filter_indices=0, seed_input=X_test[k,:,:]) 
    grads_rhi = visualize_saliency(cnn, layer_idx, filter_indices=1, seed_input=X_test[k,:,:]) 
    ax = plt.subplot(8,9,k+1)
    
    if k % 9 == 0:
        ax.set_ylabel('TB [K]')
    if k > 55:
        ax.set_xlabel('Channels')
    
    ax.plot(X_test[k,:], color='black', lw=2)
    
    ax2 = ax.twinx()
    ax2.plot(grads_fcnm, lw=3, alpha=0.8, color='blue')
    ax2.plot(grads_rhi, lw=3, alpha=0.8, color='orange')
    ax2.set_yticks([])
    ax2.axes.get_yaxis().set_visible(False) 
    
    ax.set_xlim(100,300)
    
plt.tight_layout()

In [ ]:
fcnm_zea = fits.open(fcnm_cnn_file)
rhi_zea = fits.open(rhi_cnn_file)

wcs_ngp = WCS(fcnm_zea[1].header)
wcs_sgp = WCS(fcnm_zea[3].header)

obs = Table.read(observed_catalog_file)

source_ngp = wcs_ngp.wcs_world2pix(obs['l'], obs['b'], 0)
source_sgp = wcs_sgp.wcs_world2pix(obs['l'], obs['b'], 0)

In [ ]:
def plot_cool_contours(xx, yy, kvals, ax,  find_grid=True, contourcolor=None, lww=2):

    # fit an array of size [Ndim, Nsamples]
    if contourcolor is None:
        contourcolor=tbcolor

    # evaluate on a regular grid
    if find_grid:
        data = np.vstack([xx, yy])
        kde = gaussian_kde(data)
        xgrid = np.linspace(kvals[0], kvals[1], 50)
        ygrid = np.linspace(kvals[2], kvals[3], 50)
        Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
        z = kde.evaluate(np.vstack([Xgrid.ravel(), Ygrid.ravel()]))
        z = z.reshape(Xgrid.shape)
        z = z / z.sum()
    else:
        z = xx.T
        z = z / z.sum()
        
    n = 1000
    t = np.linspace(0, z.max(), n)
    integral = ((z >= t[:, None, None]) * z).sum(axis=(1,2))

    f = interpolate.interp1d(integral, t)
    t_contours = f(np.array([0.98, 0.95, 0.68]))
    ax.contour(z, t_contours, extent=[kvals[0], kvals[1], kvals[2], kvals[3]], colors=contourcolor, zorder=1,linewidths=lww, alpha=0.8) 
    

# Figure 5

In [ ]:
fig = plt.figure(figsize=[16,6])

ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

cmapmap='BuPu'

kmin = -0.05
kmax = 1.
kvals = [kmin, kmax, kmin, kmax]

ax.errorbar(obs['fcnm'], obs['fcnm_cnn'], xerr = obs['fcnm_err'],  yerr = obs['fcnm_cnn_err'], fmt='o', markersize=6,color='darkorange', ecolor='dimgrey', alpha=1.0, zorder=4, label=r'$\rm Observed$')
ax.scatter(obs['fcnm'], obs['fcnm_cnn'], color='dimgrey', marker='o', zorder=3, s=100,alpha=1.0)
ax.grid(ls='dotted', color='grey')

plot_cool_contours(Y_validate[:,0], Y_predicted[:,0],kvals, ax, 'bone')

xvs = np.linspace(-0.1,1.,10)
ax.plot(xvs, xvs, ls='dashed', color='black')
ax.set_aspect('equal')
ax.set_ylim(-0.05,0.65)
ax.set_xlim(-0.05,0.65)

kmin = 0.96
kmax = 2.
kvals = [kmin, kmax, kmin, kmax]

ax2.errorbar(obs['rhi'], obs['rhi_cnn'], xerr = obs['rhi_err'], yerr = obs['rhi_cnn_err'], markersize=6, fmt='o', color='darkorange',  ecolor='dimgrey', alpha=1.0, zorder=4)
ax2.scatter(obs['rhi'], obs['rhi_cnn'], color='dimgrey', marker='o', zorder=3, s=100, alpha=1.0)
ax2.grid(ls='dotted', color='grey')

plot_cool_contours(Y_validate[:,1], Y_predicted[:,1], kvals, ax2, 'bone')

xvs = np.linspace(0.9,2.,10)
ax2.plot(xvs, xvs, ls='dashed',color='black')
ax2.set_aspect('equal')
ax2.set_ylim(0.96, 1.65)
ax2.set_xlim(0.96, 1.65)

ax2.set_xlabel(r'$\mathcal{R}_{\rm HI} \,\, (input)$')
ax2.set_ylabel(r'$\mathcal{R}_{\rm HI,\, CNN}$') 

ax.set_xlabel(r'$f_{\rm CNM} \,\, (input)$')
ax.set_ylabel(r'$f_{\rm CNM,\,CNN}$')

plt.tight_layout()

# Figure 7

In [ ]:
good_cmap_fcnm = plt.cm.get_cmap('Blues_r')
good_cmap_fcnm.set_bad('grey',0.)

fig = plt.figure(0,[14,11.5])
ax = plt.subplot(221, projection=wcs_ngp)
ax2 = plt.subplot(222, projection=wcs_ngp)

gridcolor='lightgray'

ax.grid(ls='dotted', color=gridcolor)
ax2.grid(ls='dotted', color=gridcolor)

fcnm_min = 0.0
fcnm_max = 0.4

greyval=0.5
greyaval=1
edgcolor='white'

im =ax.imshow(fcnm_zea[1].data, origin='lower', cmap=good_cmap_fcnm, vmin=fcnm_min, vmax=fcnm_max) #, norm=LogNorm(),) #, vmin=0, vmax=0.4)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_fcnm, c = obs['fcnm'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=fcnm_min, vmax=fcnm_max)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_fcnm, c = obs['fcnm_cnn'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=fcnm_min, vmax=fcnm_max)

ax2.imshow(fcnm_zea[3].data, origin='lower', cmap=good_cmap_fcnm, vmin=fcnm_min, vmax=fcnm_max) #, norm=LogNorm(), vmin=5e19, vmax=2e21) #, vmin=0, vmax=0.4)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_fcnm, c = obs['fcnm'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=fcnm_min, vmax=fcnm_max)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_fcnm, c = obs['fcnm_cnn'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=fcnm_min, vmax=fcnm_max)

ax.set_xlim(550,3550)
ax.set_ylim(550,3550)
ax2.set_xlim(550,3550)
ax2.set_ylim(550,3550)

plt.colorbar(im, ax=ax2, label=r'$f_{\rm CNM, CNN}$')

good_cmap_rhi = plt.cm.get_cmap('cubehelix')
good_cmap_rhi.set_bad('grey', 0.)

ax = plt.subplot(223, projection=wcs_ngp)
ax2 = plt.subplot(224, projection=wcs_ngp)

ax.grid(ls='dotted', color=gridcolor)
ax2.grid(ls='dotted', color=gridcolor)

rhi_min = 1.
rhi_max = 1.3

im =ax.imshow(rhi_zea[1].data, origin='lower', cmap=good_cmap_rhi, vmin=rhi_min, vmax=rhi_max) #, norm=LogNorm(),) #, vmin=0, vmax=0.4)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_rhi, c = obs['rhi'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=rhi_min, vmax=rhi_max)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_rhi, c = obs['rhi_cnn'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=rhi_min, vmax=rhi_max)

ax2.imshow(rhi_zea[3].data, origin='lower', cmap=good_cmap_rhi, vmin=rhi_min, vmax=rhi_max) #, vmin=0.03, vmax=0.2) #, norm=LogNorm(), vmin=5e19, vmax=2e21) #, vmin=0, vmax=0.4)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_rhi, c = obs['rhi'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=rhi_min, vmax=rhi_max)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_rhi, c = obs['rhi_cnn'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=rhi_min, vmax=rhi_max)

ax.set_xlim(550,3550)
ax.set_ylim(550,3550)
ax2.set_xlim(550,3550)
ax2.set_ylim(550,3550)

plt.colorbar(im, ax=ax2, label=r'$\mathcal{R}_{\rm HI, CNN}$')

plt.tight_layout()
plt.subplots_adjust(wspace=0)


# Figure 7b (Figure 16)

In [ ]:
fig = plt.figure(0,[14,11.5])
ax = plt.subplot(221, projection=wcs_ngp)
ax2 = plt.subplot(222, projection=wcs_ngp)

gridcolor='lightgray'

ax.grid(ls='dotted', color=gridcolor)
ax2.grid(ls='dotted', color=gridcolor)

fcnm_min=0.01
fcnm_max=0.1

greyval=0.5
greyaval=1
edgcolor='white'

im =ax.imshow(fcnm_zea[2].data, origin='lower', cmap=good_cmap_fcnm, vmin=fcnm_min, vmax=fcnm_max) #, norm=LogNorm(),) #, vmin=0, vmax=0.4)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_fcnm, c = obs['fcnm_err'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=fcnm_min, vmax=fcnm_max)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_fcnm, c = obs['fcnm_cnn_err'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=fcnm_min, vmax=fcnm_max)

ax2.imshow(fcnm_zea[4].data, origin='lower', cmap=good_cmap_fcnm, vmin=fcnm_min, vmax=fcnm_max) #, norm=LogNorm(), vmin=5e19, vmax=2e21) #, vmin=0, vmax=0.4)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_fcnm, c = obs['fcnm_err'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=fcnm_min, vmax=fcnm_max)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_fcnm, c = obs['fcnm_cnn_err'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=fcnm_min, vmax=fcnm_max)

ax.set_xlim(550,3550)
ax.set_ylim(550,3550)
ax2.set_xlim(550,3550)
ax2.set_ylim(550,3550)

plt.colorbar(im, ax=ax2, label = r'$\sigma_{f_{\rm CNM,CNN}}$')

good_cmap_rhi = plt.cm.get_cmap('cubehelix')
good_cmap_rhi.set_bad('grey', 0.)

ax = plt.subplot(223, projection=wcs_sgp)
ax2 = plt.subplot(224, projection=wcs_sgp)

ax.grid(ls='dotted', color=gridcolor)
ax2.grid(ls='dotted', color=gridcolor)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

rhi_min = 0.02
rhi_max = 0.07

im =ax.imshow(rhi_zea[2].data, origin='lower', cmap=good_cmap_rhi, vmin=rhi_min, vmax=rhi_max) #, norm=LogNorm(),) #, vmin=0, vmax=0.4)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_rhi, c = obs['rhi_err'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=rhi_min, vmax=rhi_max)
ax.scatter(source_ngp[0],source_ngp[1], cmap = good_cmap_rhi, c = obs['rhi_cnn_err'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=rhi_min, vmax=rhi_max)

ax2.imshow(rhi_zea[4].data, origin='lower', cmap=good_cmap_rhi, vmin=rhi_min, vmax=rhi_max) #, vmin=0.03, vmax=0.2) #, norm=LogNorm(), vmin=5e19, vmax=2e21) #, vmin=0, vmax=0.4)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_rhi, c = obs['rhi_err'], marker='o', edgecolor=edgcolor, s=30, lw=0.25,zorder=10, vmin=rhi_min, vmax=rhi_max)
ax2.scatter(source_sgp[0],source_sgp[1], cmap = good_cmap_rhi, c = obs['rhi_cnn_err'], marker='o', edgecolor=edgcolor, s=80, lw=0.25,zorder=9, vmin=rhi_min, vmax=rhi_max)

ax.set_xlim(550,3550)
ax.set_ylim(550,3550)

ax2.set_xlim(550,3550)
ax2.set_ylim(550,3550)

ax.set_xlabel('')
ax.set_ylabel('')
ax2.set_xlabel('')
ax2.set_ylabel('')

plt.colorbar(im, ax=ax2, label=r'$\sigma_{\mathcal{R}_{\rm HI,CNN}}$')

plt.tight_layout()
plt.subplots_adjust(wspace=0)


# Figure 8

In [ ]:
spots_obs_ngp = obs['b']>0.
spots_obs_sgp = obs['b']<0.

kvals = [-0.05,1,0.95,1.8]

fig = plt.figure(0,[12,5])
ax = plt.subplot(121)
ax2 = plt.subplot(122)

ax.set_xlim(kvals[0], 0.6)
ax.set_ylim(kvals[2],kvals[3])

ax2.set_xlim(kvals[0], 0.6)
ax2.set_ylim(kvals[2],kvals[3])

ax.scatter(obs['fcnm'][spots_obs_ngp], obs['rhi'][spots_obs_ngp], label=r'$\rm Observed$', color='darkorange', edgecolor='dimgrey', s=100, zorder=3)
ax.errorbar(obs['fcnm'][spots_obs_ngp], obs['rhi'][spots_obs_ngp], xerr = obs['fcnm_err'][spots_obs_ngp], yerr = obs['rhi_err'][spots_obs_ngp], ecolor='dimgrey', fmt='None', zorder=1)

ax2.scatter(obs['fcnm'][spots_obs_sgp], obs['rhi'][spots_obs_sgp], color='darkorange', edgecolor='dimgrey', s=100, zorder=3)
ax2.errorbar(obs['fcnm'][spots_obs_sgp], obs['rhi'][spots_obs_sgp], xerr = obs['fcnm_err'][spots_obs_sgp], yerr = obs['rhi_err'][spots_obs_sgp], ecolor='dimgrey', fmt='None', zorder=1)

H, xt, yt = np.histogram2d(np.ravel(fcnm_zea[1].data), np.ravel(rhi_zea[1].data), bins=100, range=[[kvals[0],kvals[1]],[kvals[2], kvals[3]]])
plot_cool_contours(H, H, kvals, ax, find_grid = False, contourcolor=taucolor, lww=3.5)

H, xt, yt = np.histogram2d(np.ravel(fcnm_zea[3].data), np.ravel(rhi_zea[3].data), bins=100, range=[[kvals[0],kvals[1]],[kvals[2], kvals[3]]])
plot_cool_contours(H, H, kvals, ax2, find_grid = False, contourcolor=taucolor, lww=3.5)

plot_cool_contours(Y_predicted[:,0], Y_predicted[:,1], kvals, ax, lww=2) 
plot_cool_contours(Y_predicted[:,0], Y_predicted[:,1], kvals, ax2, lww=2)

ax.set_xlabel(r'$f_{\rm CNM}$')
ax.set_ylabel(r'$\mathcal{R}_{\rm HI}$')
ax2.set_xlabel(r'$f_{\rm CNM}$')
ax2.set_ylabel(r'$\mathcal{R}_{\rm HI}$')

ax.text(0.06, 0.93,r"$\rm (a)$",
            horizontalalignment='center',
            verticalalignment='center',
            transform=ax.transAxes, fontsize=16)  
ax2.text(0.06, 0.93,r"$\rm (b)$",
            horizontalalignment='center',
            verticalalignment='center',
            transform=ax2.transAxes, fontsize=16)  

ax.grid(ls='dotted', color='darkgrey')
ax2.grid(ls='dotted', color='darkgrey')

# Hack for an easy legend
ax.plot([12,12],[13,13], color=tbcolor, lw=2, label=r'$\rm CNN:\, Synthetic$') #cmap=cmapp)
ax.plot([12,12],[13,13], color=taucolor, lw=3.5, label=r'$\rm CNN:\, GALFA-HI$')
ax.legend(loc='upper right', fontsize=12)

plt.tight_layout()